In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
data_file = pd.read_csv('data/landingData.csv')
column_names = ['pos_z','index']
feature_names = column_names[0]
label_name = ['index']
print("Feature: {}".format(feature_names))
print("Label: {}".format(label_name))

In [ ]:
Vx_train, Vx_test, labelVx_train, labelVx_test = train_test_split(data_file[feature_names], data_file[label_name[0]], test_size=0.1)

In [ ]:
modelVx = keras.Sequential([
        keras.layers.Dense(16, activation=tf.nn.relu,input_shape=(1,)),
        keras.layers.Dense(32,activation=tf.nn.relu),
        keras.layers.Dense(32,activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])
optimizer = tf.keras.optimizers.RMSprop(0.001)
modelVx.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae']             
             )
modelVx.summary()

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel("Mean Abs Error [1000$]")
    plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
            label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
            label='Val Loss')
    plt.legend()
    plt.ylim([0,5])

In [ ]:
EPOCHS = 60
history=modelVx.fit(Vx_train,
                  labelVx_train,
                  epochs=EPOCHS,
                  validation_split=0.1,
                  verbose=1
                 )
plot_history(history)

In [ ]:
[loss, mae] = modelVx.evaluate(Vx_test, labelVx_test, verbose=1)
print("Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

In [ ]:
modelVx.save("models/modelVerticalNN.h5")